In [38]:
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

In [43]:
## load songs
## each song is a list of tuples (chords)
with open("songs_final_2.pkl" , "rb") as f:
    data = pickle.load(f) 

In [44]:
data[0]

['F-1_A3_E4_A4_C#5_E6',
 'E-1_B--1_G#3_F4_B-4',
 'A-1_F4_B-4',
 'A-1_B--1_G3_F4_F#4_B-4_B4',
 'A-1_B--1_G3_F4_F#4_B-4_B4',
 'A-1_F#4_B4',
 'A-1_B--1_F#3_F#4_G4_B4_C5',
 'A-1_G4_C5',
 'A-1_B--1_F3_G4_G#4_C5_C#5',
 'A-1_B--1_F3_G4_G#4_C5_C#5',
 'A-1_G#4_C#5',
 'A-1_B--1_F3_A3_G#4_A4_C#5_D5',
 'A4_D5',
 'A-1_B--1_E-3_A4_B-4_D5_E-5',
 'A-1_B--1_E-3_E3_A4_B-4_B4_D5_E-5_E5',
 'A-1_B--1_E3_B4_E5',
 'A-1_B--1_E3_B4_E5',
 'F-1_A-1_E3_B4_E5',
 'F-1_A-1_B4_E5',
 'E-1_B4_E5',
 'E-1_B--1_E3_B4_E5',
 'E-1_B--1_E3_B-4_B4_E5',
 'E3_B-4_B4_E5',
 'A-1_B--1_E3_B-4_B4_E5',
 'F-1_A-1_B--1_E3_B-4_B4_E5',
 'F-1_A-1_E3_B-4_B4_E5',
 'E-1_B-4_B4_E5',
 'E-1_B--1_E3_B-4_E5',
 'E-1_B--1_E3_B-4_B4_E5',
 'A-1_B--1_E3_B-4_B4_E5',
 'A-1_B--1_E3_B-4_B4_E5',
 'F-1_A-1_B--1_E3_B-4_B4_E5',
 'E-1_F-1_A-1_E3_B-4_B4_E5',
 'E-1_B-4_B4_E5',
 'E-1_B--1_E3_B4_E5',
 'A-1_B--1_E3_B4_E5',
 'F-1_A-1_E3_B4_E5',
 'E-1_F-1_A-1_B4_E5',
 'E-1_B4_E5',
 'B4_E5',
 'E-1_A-1_B--1_D3_E3_B4_E5',
 'E-1_A-1_B--1_D3_E3_E5',
 'A-1_C3_B-4_B4_E5',
 '

In [45]:
# organize into sequences of tokens
length = 4 + 1
lines = list()
vocab = []
for song in data:
    for i in range(length, len(song)):
        # select sequence of tokens
        vocab+=song
        seq = song[i-length:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        lines.append(line)
    print('Total Sequences: %d' % len(lines))
vocab = list(set(vocab))

Total Sequences: 476
Total Sequences: 477
Total Sequences: 481
Total Sequences: 933
Total Sequences: 999
Total Sequences: 1173
Total Sequences: 1528
Total Sequences: 1704
Total Sequences: 1720
Total Sequences: 1725
Total Sequences: 1791
Total Sequences: 2055
Total Sequences: 2087
Total Sequences: 2087
Total Sequences: 2258
Total Sequences: 2375
Total Sequences: 2441
Total Sequences: 2940
Total Sequences: 3436
Total Sequences: 3441
Total Sequences: 3966
Total Sequences: 4180
Total Sequences: 4197
Total Sequences: 4508
Total Sequences: 4556
Total Sequences: 5413
Total Sequences: 5500
Total Sequences: 5665
Total Sequences: 6172
Total Sequences: 6371
Total Sequences: 6417
Total Sequences: 6449
Total Sequences: 6713
Total Sequences: 6718
Total Sequences: 6911
Total Sequences: 7687
Total Sequences: 7757
Total Sequences: 8023
Total Sequences: 8037
Total Sequences: 8325
Total Sequences: 8792
Total Sequences: 9225
Total Sequences: 9375
Total Sequences: 9529
Total Sequences: 9817
Total Sequences

Total Sequences: 124197
Total Sequences: 124282
Total Sequences: 124452
Total Sequences: 124612
Total Sequences: 125138
Total Sequences: 125402
Total Sequences: 125891
Total Sequences: 126050
Total Sequences: 126157
Total Sequences: 126332
Total Sequences: 126338
Total Sequences: 126537
Total Sequences: 127107
Total Sequences: 127500
Total Sequences: 127868
Total Sequences: 127902
Total Sequences: 127913
Total Sequences: 128150
Total Sequences: 129078
Total Sequences: 129216
Total Sequences: 129216
Total Sequences: 129435
Total Sequences: 129584
Total Sequences: 130458
Total Sequences: 130511
Total Sequences: 130741
Total Sequences: 130838
Total Sequences: 130899
Total Sequences: 131032
Total Sequences: 131047
Total Sequences: 131386
Total Sequences: 131454
Total Sequences: 131478
Total Sequences: 131763
Total Sequences: 131807
Total Sequences: 131830
Total Sequences: 131867
Total Sequences: 132830
Total Sequences: 132833
Total Sequences: 133560
Total Sequences: 133724
Total Sequences:

In [49]:
## mappings from chords to unique integers and the reverse
chord2ind = {chord : i for i,chord in enumerate(vocab)}
ind2chord = {i : chord for i,chord in enumerate(vocab)}

In [50]:
def convert_line(line, dictio):
    """converts a sequence of chords(str) into list of integers"""
    return [dictio[el] for el in line.split(" ")]

In [51]:
# integer encode sequences of words
sequences = [convert_line(line, chord2ind) for line in lines]

In [53]:
sequences[0]

array([81272, 36815, 31758, 46064, 62605])

In [52]:
## create inputs and outputs
sequences = np.array([np.array(seq) for seq in sequences])
np.random.seed(42)
np.random.shuffle(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=len(vocab))
seq_length = X.shape[1]

MemoryError: Unable to allocate 71.9 GiB for an array with shape (225420, 85570) and data type float32

In [ ]:
X.shape

In [ ]:
vocab

In [ ]:
# define model
model = Sequential()
model.add(Embedding(len(vocab), 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(vocab), activation='softmax'))
print(model.summary())

In [29]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=2)

Epoch 1/2
111650/111650 [==============================] - 14s 125us/sample - loss: 0.2432 - acc: 0.9446
Epoch 2/2
111650/111650 [==============================] - 14s 124us/sample - loss: 0.0041 - acc: 0.9997


In [30]:
def generate_seq(model, chord2ind, ind2chord, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = convert_line(in_text, chord2ind)
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ind2chord[int(yhat)]
		
		# append to input
		in_text += ' ' + out_word.upper()
		result.append(out_word)
	return ' '.join(result)

In [31]:
# save the model to file
model.save('model_2.h5')
# save the tokenizer
with open("chord2ind_2.pkl", "wb") as f:
    pickle.dump(chord2ind, f)
with open("ind2chord_2.pkl", "wb") as f:
    pickle.dump(ind2chord, f)

In [35]:
seed_prog = np.random.choice(lines)

In [36]:
example = generate_seq(model, chord2ind, ind2chord,seq_length, seed_prog, 200)

In [37]:
example

'A3_E4_A4_C#5 B3_D4_F#4_B4 A2_C#4_E4_A4 F#3_B3_F#4_B4 A3_B3_E4_B4 F#2_A3_C#4_F#4 E#3_C#4_G#4_C#5 A3_E4_C#5 B2_A3_D#4_F#4 C#3_G#3_C#4_E4 B2_D4_G#4_B4 D3_F#3_B3_F#4 D4_F#4_A4 G#3_B3_E4_B4 B2_B3_D4_E#4 B3_D#4_F#4 F#3_C#4_F#4_A4 B2_B3_D4_F#4 A#2_F#3_D4_F#4 F#3_C#4_A4 C#3_C#4_E#4_G#4 G#3_B3_F#4_B4 B2_D4_F#4_G#4 C#3_B3_D4_F#4 F#3_A#3_C#4_F#4 A#2_C#4_E4_F#4 F#3_D4_F#4_A4 B2_C#4_E4_F#4 A#2_F#3_C#4_F#4 C#3_E4_A4_C#5 C#3_A#3_C#4_F#4 E3_E4_G#4_B4 D3_B3_F#4 D3_B3_F#4 D3_B3_F#4 G#3_B3_F#4_B4 C#3_E4_A4_C#5 B2_A3_D#4_F#4 F#3_A#3_C#4_F#4 A3_B3_E4_B4 E3_E4_G#4_B4 D4_F#4_A4 G#3_B3_E4_B4 B2_A3_D#4_F#4 G#3_B3_F#4_B4 B3_D#4_F#4 B2_A3_D#4_F#4 D3_F#3_B3_F#4 A#2_C#4_E4_F#4 G#3_B3_E4_B4 D4_F#4_A4 B3_D#4_F#4 B3_D#4_F#4 F#3_C#4_F#4_A4 F#3_C#4_F#4_A4 F#3_C#4_A4 F#3_C#4_A4 C#3_C#4_E#4_G#4 C#3_C#4_E#4_G#4 B2_D4_F#4_G#4 C#3_B3_D4_F#4 F#3_C#4_A4 F#3_A#3_C#4_F#4 F#3_D4_F#4_A4 B2_A3_D#4_F#4 C#3_B3_D4_F#4 A#2_F#3_C#4_F#4 G#3_B3_E4_B4 D3_F#3_B3_F#4 B2_C#4_E4_F#4 C#3_B3_D4_F#4 B3_D#4_F#4 G#3_B3_E4_B4 F#3_D4_F#4_A4 B2_C#4_